In [ ]:
# Paramètres injectés par papermill
MLFLOW_URL = "https://e06b-41-248-47-247.ngrok-free.app/"
PUSHGATEWAY_URL = "https://e546-160-179-231-239.ngrok-free.app/"
output_dir = "output"
file_path = "lstm_predictions_charger.csv"

In [ ]:
import os
import json

# Vérifier que le dossier output existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Définir le chemin du modèle
model_path = os.path.join(output_dir, "ppo_bess_model")

# Exécuter train_ppo.py
train_command = f"python train_ppo.py --file_path {file_path} --mlflow_url {MLFLOW_URL} --output_dir {output_dir} --model_path {model_path}"
os.system(train_command)

# Exécuter evaluate_ppo.py
eval_command = f"python evaluate_ppo.py --file_path {file_path} --model_path {model_path} --mlflow_url {MLFLOW_URL} --output_dir {output_dir}"
os.system(eval_command)

# Charger et pousser les métriques vers Prometheus
PUSHGATEWAY_URL = os.getenv("PUSHGATEWAY_URL", PUSHGATEWAY_URL)

# Métriques d'entraînement
train_metrics_path = os.path.join(output_dir, "ppo_bess_model_metrics.json")
if os.path.exists(train_metrics_path):
    with open(train_metrics_path, "r") as f:
        train_metrics = json.load(f)
    for key, value in train_metrics.items():
        os.system(f'echo "{key}{{run=\\\"ppo_training\\\"}} {value}" | curl --data-binary @- {PUSHGATEWAY_URL}/metrics/job/ppo_metrics')

# Métriques d'évaluation
eval_metrics_path = os.path.join(output_dir, "evaluation_metrics.json")
if os.path.exists(eval_metrics_path):
    with open(eval_metrics_path, "r") as f:
        eval_metrics = json.load(f)
    for key, value in eval_metrics.items():
        os.system(f'echo "{key}{{run=\\\"ppo_evaluation\\\"}} {value}" | curl --data-binary @- {PUSHGATEWAY_URL}/metrics/job/ppo_metrics')

# Pousser une métrique de succès
os.system(f'echo "success 1" | curl --data-binary @- {PUSHGATEWAY_URL}/metrics/job/ppo_metrics')